<a href="https://colab.research.google.com/github/rsmecking/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/Ryan_Mecking_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
print (df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
#Encoding
from sklearn.preprocessing import OrdinalEncoder 
enc = OrdinalEncoder()

categorical = df.select_dtypes(include='object')
#total charges was encoding and didn't really need to be.
# categorical = categorical.drop('TotalCharges', axis=1)

df[categorical.columns] = enc.fit_transform(df[categorical.columns])
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375.0,0.0,0,1.0,0.0,1,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,29.85,2505.0,0.0
1,3962.0,1.0,0,0.0,0.0,34,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,56.95,1466.0,0.0
2,2564.0,1.0,0,0.0,0.0,2,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,53.85,157.0,1.0
3,5535.0,1.0,0,0.0,0.0,45,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,42.30,1400.0,0.0
4,6511.0,0.0,0,0.0,0.0,2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,70.70,925.0,1.0


In [28]:
#train/test split
import numpy as np
from sklearn.model_selection import train_test_split

df1 = df.drop(['Churn'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(df, df['Churn'], test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4718, 21), (2325, 21), (4718,), (2325,))

In [29]:
X_train.dtypes

customerID          float64
gender              float64
SeniorCitizen         int64
Partner             float64
Dependents          float64
tenure                int64
PhoneService        float64
MultipleLines       float64
InternetService     float64
OnlineSecurity      float64
OnlineBackup        float64
DeviceProtection    float64
TechSupport         float64
StreamingTV         float64
StreamingMovies     float64
Contract            float64
PaperlessBilling    float64
PaymentMethod       float64
MonthlyCharges      float64
TotalCharges        float64
Churn               float64
dtype: object

In [30]:
#Scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train[:1])

[[-0.71046684  0.96869657 -0.43441055  1.02925348  1.51254913  0.30741281
   0.32271084  1.10463441 -1.1831419  -0.92416281 -1.04065084  1.26021011
  -0.92993691  1.13899081  1.12746159 -0.83859456  0.84043388  0.40414053
   0.32802721 -0.35309092 -0.59742521]]


In [34]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 5
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(21,)))
model.add(Dense(64, activation='relu'))

# Compile Model
model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Epoch 1/5
472/472 [==============================] - 2s 5ms/step - loss: 0.9701 - accuracy: 0.0199 - val_loss: 0.2734 - val_accuracy: 0.0228
Epoch 2/5
472/472 [==============================] - 2s 4ms/step - loss: 0.1860 - accuracy: 0.1327 - val_loss: 0.1505 - val_accuracy: 0.2271
Epoch 3/5
472/472 [==============================] - 2s 4ms/step - loss: 0.1369 - accuracy: 0.3152 - val_loss: 0.1355 - val_accuracy: 0.3935
Epoch 4/5
472/472 [==============================] - 2s 4ms/step - loss: 0.1298 - accuracy: 0.4879 - val_loss: 0.1323 - val_accuracy: 0.5303
Epoch 5/5
472/472 [==============================] - 2s 4ms/step - loss: 0.1281 - accuracy: 0.5922 - val_loss: 0.1312 - val_accuracy: 0.6228


In [16]:
# fix random seed for reproducibility
import numpy 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Nadam

seed = 42
numpy.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(learning_rate=.01):
    # create model
    model = Sequential()
    model.add(Dense(19, activation='relu'))
    model.add(Dense(1, activation='relu'))
    # Compile model
    optimizer = Nadam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'learning_rate': [.001, .01, .1, .2, .3, .5],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 1.0 using {'epochs': 20, 'learning_rate': 0.01}
Means: 0.9815536379814148, Stdev: 0.02284831464278241 with: {'epochs': 20, 'learning_rate': 0.001}
Means: 1.0, Stdev: 0.0 with: {'epochs': 20, 'learning_rate': 0.01}
Means: 0.9983039617538452, Stdev: 0.0021831371236631174 with: {'epochs': 20, 'learning_rate': 0.1}
Means: 0.8739298939704895, Stdev: 0.11359553492551391 with: {'epochs': 20, 'learning_rate': 0.2}
Means: 0.7369679808616638, Stdev: 0.010528864323655737 with: {'epochs': 20, 'learning_rate': 0.3}
Means: 0.7369679808616638, Stdev: 0.010528864323655737 with: {'epochs': 20, 'learning_rate': 0.5}


In [18]:
# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 20
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(19, activation='relu', input_shape=(inputs,)))
model.add(Dense(2, activation='relu'))

# Compile Model
model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Epoch 1/20
472/472 [==============================] - 2s 5ms/step - loss: 2.3779 - accuracy: 0.5551 - val_loss: 1.7200 - val_accuracy: 0.5583
Epoch 2/20
472/472 [==============================] - 2s 5ms/step - loss: 1.3132 - accuracy: 0.6043 - val_loss: 0.9179 - val_accuracy: 0.5892
Epoch 3/20
472/472 [==============================] - 2s 4ms/step - loss: 0.8563 - accuracy: 0.6159 - val_loss: 0.6058 - val_accuracy: 0.6370
Epoch 4/20
472/472 [==============================] - 2s 4ms/step - loss: 0.4414 - accuracy: 0.6547 - val_loss: 0.2509 - val_accuracy: 0.6413
Epoch 5/20
472/472 [==============================] - 2s 4ms/step - loss: 0.2614 - accuracy: 0.6744 - val_loss: 0.1629 - val_accuracy: 0.6804
Epoch 6/20
472/472 [==============================] - 2s 5ms/step - loss: 0.1476 - accuracy: 0.6942 - val_loss: 0.0937 - val_accuracy: 0.6809
Epoch 7/20
472/472 [==============================] - 2s 4ms/step - loss: 0.0878 - accuracy: 0.7187 - val_loss: 0.0610 - val_accuracy: 0.7290
Epoch 

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?